## FASE 1: EXPLORACIÓN Y LIMPIEZA

### 1. Exploración inicial: 

- Realiza una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes.

- Utiliza funciones de Pandas para obtener información sobre la estructura de los datos, la presencia de valores nulos y estadisticas básicas de las columnas involucradas.

- Une los dos conjuntos de datos de la forma más eficiente.

### 2. Limpieza de datos:

- Elimina o trata los valores nulos, si los hay, en las columnas clave para asegurar que los datos estén completos.

- Verifica la consistencia y corrección de los datos para asegurarte de que los datos se presenten de forma coherente.

- Realiza cualquier ajuste o conversión necesaria en las columnas (por ejemplo, cambiar tipos de datos) para garantizar la adecuación de los datos para el análisis estadístico.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
# Aplico una función para leer los datos del fichero:
def leer_datos(ruta, sep=","):

    df = pd.read_csv(ruta, sep=sep)
    return df

# Para hacer la llamada de la función
df_customer_flight=leer_datos('customer_flight_activity.csv', sep=";")
print(df_customer_flight)

       Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0                        100018,2017,1,3,0,3,1521,152,0,0                                                                                                    
1                      100102,2017,1,10,4,14,2030,203,0,0                                                                                                    
2                        100140,2017,1,6,0,6,1200,120,0,0                                                                                                    
3                             100214,2017,1,0,0,0,0,0,0,0                                                                                                    
4                             100272,2017,1,0,0,0,0,0,0,0                                                                                                    
...                                                 

**Abriendo el fichero customer flight activity, me doy cuenta de que tiene todas las variables en una única columna.**

**Hay que separarlas, usando la coma cómo separador de columnas.**

In [4]:
#Voy a dividir la columna en 10 columnas, utilizando las comas para dividir las columnas
df_split_flight = df_customer_flight[
    'Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed'
    ].str.split(',', expand=True) 

# Voy a asignar los nombres a las columnas:
df_split_flight.columns = [
    'Loyalty Number', 'Year', 'Month', 'Flights Booked', 'Flights with Companions',
    'Total Flights', 'Distance', 'Points Accumulated', 'Points Redeemed',
    'Dollar Cost Points Redeemed'
]

# Con head visualizo las primeras 5 filas de mi nuevo dataframe
df_split_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152,0,0
1,100102,2017,1,10,4,14,2030,203,0,0
2,100140,2017,1,6,0,6,1200,120,0,0
3,100214,2017,1,0,0,0,0,0,0,0
4,100272,2017,1,0,0,0,0,0,0,0


In [3]:
def leer_datos(ruta, sep=","):

    df = pd.read_csv(ruta, sep=sep)
    return df

# Para hacer la llamada de la función
df_customer_loyalty=leer_datos('customer_loyalty_history.csv', sep=";")
print(df_customer_loyalty)

      Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0      480934,Canada,Ontario,Toronto,M2Z 4K1,Female,B...                                                                                                                                            
1      549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,Co...                                                                                                                                            
2      429460,Canada,British Columbia,Vancouver,V6E 3...                                                                                                                                            
3      608370,Canada,Ontario,Toronto,P1W 1K4,Male,Col...                                                                                                                                            
4      530508,C

**Abriendo el fichero customer loyalty history, me doy cuenta que tenemos el mismo problema del otro fichero, voy a dividir las columnas con el mismo metodo.**

In [5]:
#Voy a dividir la columna en 16 columnas, utilizando las comas para dividir las columnas
df_split_loyalty = df_customer_loyalty['Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month'].str.split(',', expand=True) 

# Voy a asignar los nombres a las columnas:
df_split_loyalty.columns = [
   'Loyalty Number', 'Country', 'Province', 'City', 'Postal Code', 'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card', 'CLV' , 'Enrollment Type', 'Enrollment Year', 'Enrollment Month', 'Cancellation Year', 'Cancellation Month'
]

df_split_loyalty.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236,Married,Star,3839.14,Standard,2016,2,,
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,,Divorced,Star,3839.61,Standard,2016,3,,
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,,Single,Star,3839.75,Standard,2014,7,2018,1
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,,Single,Star,3839.75,Standard,2013,2,,
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495,Married,Star,3842.79,Standard,2014,10,,


**Ahora que tengo los ficheros divididos en columnas, voy a analizarlos con una función para comprobar si hay valores nulos, duplicados, atípicos o datos faltantes.**

In [7]:
def explorar_dataset(df): # Función generica para una visión general del dataset

    print("Tamaño del dataset:")# Tamaño del dataset (filas y columnas)
    print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}") # Tamaño del dataset (filas y columnas)
    print("\nTipos de datos:") # Tipos de datos por columna
    display(df.dtypes)
    print("\nValores_nulos:") 
    display(df.isnull().sum())
    print("Filas duplicadas en el dataset:")
    print(f"Filas: {df.duplicated().sum()}") #calcula la cantidad de filas duplicadas en el df

# Llamamos a la función para ver los datos de cadauno de los df
print(f'Dataframe actividad vuelos clientes:')
explorar_dataset(df_split_flight)
print('_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _', '\n')
print(f'Dataframe historial clientes:')
explorar_dataset(df_split_loyalty)


Dataframe actividad vuelos clientes:
Tamaño del dataset:
Filas: 405624, Columnas: 10

Tipos de datos:


Loyalty Number                 object
Year                           object
Month                          object
Flights Booked                 object
Flights with Companions        object
Total Flights                  object
Distance                       object
Points Accumulated             object
Points Redeemed                object
Dollar Cost Points Redeemed    object
dtype: object


Valores_nulos:


Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

Filas duplicadas en el dataset:
Filas: 1864
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

Dataframe historial clientes:
Tamaño del dataset:
Filas: 16737, Columnas: 16

Tipos de datos:


Loyalty Number        object
Country               object
Province              object
City                  object
Postal Code           object
Gender                object
Education             object
Salary                object
Marital Status        object
Loyalty Card          object
CLV                   object
Enrollment Type       object
Enrollment Year       object
Enrollment Month      object
Cancellation Year     object
Cancellation Month    object
dtype: object


Valores_nulos:


Loyalty Number        0
Country               0
Province              0
City                  0
Postal Code           0
Gender                0
Education             0
Salary                0
Marital Status        0
Loyalty Card          0
CLV                   0
Enrollment Type       0
Enrollment Year       0
Enrollment Month      0
Cancellation Year     0
Cancellation Month    0
dtype: int64

Filas duplicadas en el dataset:
Filas: 0


In [8]:
# Mostrar las filas duplicadas del df de actividad de vuelos:
df_flight_duplicates = df_split_flight[df_split_flight.duplicated()]
print(df_flight_duplicates)

       Loyalty Number  Year Month Flights Booked Flights with Companions  \
42             101902  2017     1              0                       0   
227            112142  2017     1              0                       0   
478            126100  2017     1              0                       0   
567            130331  2017     1              0                       0   
660            135421  2017     1              0                       0   
...               ...   ...   ...            ...                     ...   
404668         949628  2018    12              0                       0   
404884         960050  2018    12              0                       0   
405111         971370  2018    12              0                       0   
405410         988392  2018    12              0                       0   
405437         989528  2018    12              0                       0   

       Total Flights Distance Points Accumulated Points Redeemed  \
42                 

**Observando estas informaciones, puedo ver ya que hay que cambiar algunos tipos de datos en los dos df, porque son todos object, o sea categoricos y algunos necesito que sean numericos para poder calcular porcentajes y estadisticas, como por ejemplo, en el df de la actividad de vuelos: Flights Booked, Flights with Companions, Total Flights, Distance, Points Accumulated, Points Redeemed, Dollar Cost Points Redeemed.**

**También veo que hay 1864 filas duplicadas en el dataset de la actividad de vuelos, pero he comprobado y veo que se trata de años, meses, numeros de puntos acumulados y redimidos, el valor en dolares de los puntos, etc. No hace falta modificar esos datos**


In [ ]:
# Voy a transformar el tipo de datos da categoricos a numerico en las columnas que he identificado antes:

def cambiar_tipo_datos(df):
    # Diccionario que mapea las columnas a su tipo deseado
    tipos_datos = {
        'Flights Booked': 'float64', 
        'Flights with Companions': 'float64', 
        'Total Flights': 'float64', 
        'Distance': 'float64', 
        'Points Accumulated': 'float64', 
        'Points Redeemed': 'float64', 
        'Dollar Cost Points Redeemed': 'float64'
    }
    
    # Aplicar la conversión de tipos de datos
    df = df.astype(tipos_datos)

    return df

# Usar la función en tu DataFrame
df_split_flight = cambiar_tipo_datos(df_split_flight)

# Verificar los tipos de datos después del cambio
print(df_split_flight.dtypes)


In [12]:
# Ahora quiero estandarizar los nombres de todas las columnas de los dos df:
# cambiar todas las etiquetas a minusculas y sustituir los espacios con guion bajo:

df_split_flight.columns = [col.lower().replace(" ", "_") for col in df_split_flight.columns] #df actividad vuelos
print(f'Primeras 5 filas df Actividad vuelos:')
print(df_split_flight.head()) 

print('_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _', '\n')

df_split_loyalty.columns = [col.lower().replace(" ", "_") for col in df_split_loyalty.columns] #df historial clientes
print(f'Primeras 5 filas df Historial clientes:')
print(df_split_flight.head()) 

Primeras 5 filas df Actividad vuelos:
  loyalty_number  year month  flights_booked  flights_with_companions  \
0         100018  2017     1             3.0                      0.0   
1         100102  2017     1            10.0                      4.0   
2         100140  2017     1             6.0                      0.0   
3         100214  2017     1             0.0                      0.0   
4         100272  2017     1             0.0                      0.0   

   total_flights  distance  points_accumulated  points_redeemed  \
0            3.0    1521.0               152.0              0.0   
1           14.0    2030.0               203.0              0.0   
2            6.0    1200.0               120.0              0.0   
3            0.0       0.0                 0.0              0.0   
4            0.0       0.0                 0.0              0.0   

   dollar_cost_points_redeemed  
0                          0.0  
1                          0.0  
2                    

In [14]:
# Voy a transformar otra vez el tipo de datos da float a int en algunas de las columnas que he transformado antes:

def cambiar_tipo_datos(df):
    # Diccionario que mapea las columnas a su tipo deseado
    tipos_datos = {
        'flights_booked': 'int64', 
        'flights_with_companions': 'int64', 
        'total_flights': 'int64', 
        'points_accumulated': 'int64', 
        'points_redeemed': 'int64', 
    }
    
    # Aplicar la conversión de tipos de datos
    df = df.astype(tipos_datos)

    return df

# Usar la función en tu DataFrame
df_split_flight = cambiar_tipo_datos(df_split_flight)

# Verificar los tipos de datos después del cambio
print(df_split_flight.dtypes)


loyalty_number                  object
year                            object
month                           object
flights_booked                   int64
flights_with_companions          int64
total_flights                    int64
distance                       float64
points_accumulated               int64
points_redeemed                  int64
dollar_cost_points_redeemed    float64
dtype: object


**Ahora hay que cambiar los meses, o sea sustituir al numero 1 la palabra 'enero', al 2 la palabra 'febrero' y así sucesivamente.**

In [ ]:
# Cambiar meses en columna month:

## FASE 2: VISUALIZACIÓN 

### Usando las herramientas de visualización que has aprendido durante el módulo, contesta a las siguientes preguntas usando la mejor gráfica que consideres:

***1. ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?***

***2. ¿Existe una relación entre la distancia de los vuelos y los puntos acumulados por los clientes?***

***3. ¿Cuál es la distribución de los clientes por provincia o estado?***

***4. ¿Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?***

***5. ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?***

***6. ¿Cómo se distribuyen los clientes según su estado civil y género?***